# Exploring Graphs of Even and Odd Paths
-----

## Table of Contents

* [Model Setup](#Model-Setup)
    * [Model Introduction](#Model-Introduction)
    * [Dependencies](#Dependencies)
* [Model Design](#Model-Design)
    * [Procedural Graph Generator](#Procedural-Graph-Generator)
    * [Define the Genetic Algorithm Skeleton](#Define-the-Genetic-Algorithm-Skeleton)
    * [Automation: Defining the Simulation Suite](#Automation:-Defining-the-Graph-Simulation-Suite)
* [Simulations](#Simulations)
    * [Odd Paths](#Odd-Paths)
    * [Even Paths](#Even-Paths)
* [Conclusions](#Conclusions)
    * [Odd Path Review](#Odd-Paths-Review)
    * [Even Path Review](#Even-Paths-Review)

---
# Model Setup
----

## Model Introduction

In order to build a decent pool of data for us to collect information and derive possible schemas for coloring, investigating the differences between Odd and Even paths is critical. We will first analyze Odd paths and then even and wrap it all up in the [conclusions](#Conclusions) section.

## Dependencies

Grouping our simulation dependencies together for neatness and organization.

In [1]:
import sys
sys.path.append("../..")

from classes.genetic_algorithm import GeneticAlgorithm
from graph_coloring.classes.gc_ruleset import GCRuleset
from graph_coloring.classes.gc_random_init_strategy import GCRandomInitStrategy

----
# Model Design
---

In order for these simulations to be run in an automated fashion, let's define a factory function to build out graphs based on a given number of vertices.

## Procedural Graph Generator

In [2]:
def generate_path(num_vertices: int):
    """
    Generates a path with the provided number of vertices
    
    Parameters:
        num_vertices: the number of vertices in the path
    
    Returns:
        A path of a specified length
    
    """
    path = []
    
    for v in range(num_vertices):
        if v == 0:
            path.append({"color": 0, "adj": [v + 1]})

        elif v == num_vertices - 1:
            path.append({"color": 0, "adj": [v - 1]})

        else:
            path.append({"color": 0, "adj": [v - 1, v + 1]})
    return path

In [3]:
generate_path(5)

[{'color': 0, 'adj': [1]},
 {'color': 0, 'adj': [0, 2]},
 {'color': 0, 'adj': [1, 3]},
 {'color': 0, 'adj': [2, 4]},
 {'color': 0, 'adj': [3]}]

Now let's define our Genetic Algorithm object, where the parameters are:

    - `ruleset` - The current ruleset being tested
    - `random_on_init_strategy`
    - `strat_data`

The last two will determine how each member of the population gets their respective strategey. The only two major aspects we are concerned with for this simulation are the *number of vertices in the path* and the *number of colors* ($k$) in the game.

## Define the Genetic Algorithm Skeleton

In [4]:
def path_test(num_vertices: int, k_colors: int):
    """
    Performs an evolutionary algorithm on a Path graph with the provided number of vertices and colors.
    
    Parameters:
        num_vertices: Order of the path
        k_colors: Number of colors for this game
        
    Returns:
        Two populations of players after the evolution
    """
    # Define the Path
    initial_state = generate_path(num_vertices)
    
    # Define the Ruleset of the game
    ruleset = GCRuleset("Graph Coloring Ruleset", initial_state, bounds = k_colors)
    
    # Create a new Evolution instance with the example strategy
    gen_algo = GeneticAlgorithm(
        ruleset,
        # Random-on-initialization strategy for generating populations of random players
        random_on_init_strat = GCRandomInitStrategy,
        # Data to be used by the above strategy
        strat_data = {"vertices": range(len(ruleset.initial_state)), "colors": range(1, ruleset.bounds + 1)},
        # Size of the populations
        pop_size = 100,
        # Number of generations to iterate through
        iterations = 10,
        # Minimum number of games each player must play during a generation
        num_games = 10,
        # Starting fitness threshold
        fitness = 0.5,
        # Maximum fitness threshold; be careful of setting this too close to 1.0
        max_fitness = 0.9,
        # How much the fitness threshold should increment after each iteration
        fitness_increment = 0.025,
        # Chance of a mutation to occur during player reproduction
        mutation_rate = 0.025
    )

    p1_pop, p2_pop = gen_algo.evolve(to_df=True)
    
    return p1_pop, p2_pop


## Automation: Defining the Graph Simulation Suite

In [5]:
def graph_simulation_suite(orders, k_colors, graph_test):
    """
    Runs a series of simulations on the orders and k_colors
    
    Parameters:
        orders: A list of graph orders to test
        k_colors: A list of colors that each game might have
        graph_test: A function that will take the params and run the simulations
    
    Returns:
        simulations: A dictionary containing an entry for each order and the respective 
                     k colorings winning player populations
    """
    simulations = {}
    for order in orders:
        current_run = {}
        print("----------------------------------------------------------------------")
        for k in k_colors:
            print("\n\tTesting Graph of Order " + str(order) + " with k = " + str(k))
            p1_pop, p2_pop = graph_test(num_vertices = order, k_colors = k)
            print(p1_pop.head())
            print(p2_pop.head())
            current_run[str(k)] = (p1_pop, p2_pop)
        simulations[str(order)] = current_run
        print("----------------------------------------------------------------------")
    return simulations


----
# Simulations
----

## Odd Paths

Now to run a series of experiments and prepare to analyze the data. Where the number of colors is $k = 3$.

In [6]:
p1_pop, p2_pop = path_test(num_vertices = 5, k_colors = 3)

In [7]:
p1_pop.head()

,Name,Gen,Vertices,Colors,Fitness,Wins,Losses
85,Player 1,0,"[0, 2, 4, 1, 3]","[2, 3, 1]",1.0,233,0
77,Player 1,0,"[4, 2, 1, 3, 0]","[1, 3, 2]",1.0,221,0
87,Player 1,0,"[2, 3, 1, 0, 4]","[2, 1, 3]",1.0,221,0
39,Player 1,0,"[4, 1, 2, 0, 3]","[2, 3, 1]",1.0,218,0
50,Player 1,0,"[1, 4, 3, 2, 0]","[3, 2, 1]",1.0,217,0


In [8]:
p2_pop.head()

,Name,Gen,Vertices,Colors,Fitness,Wins,Losses
0,Player 2,0,"[0, 2, 4, 1, 3]","[2, 3, 1]",0.0,0,0
1,Player 2,0,"[0, 3, 2, 1, 4]","[3, 2, 1]",0.0,0,0
2,Player 2,0,"[1, 3, 0, 4, 2]","[1, 3, 2]",0.0,0,0
3,Player 2,0,"[1, 3, 2, 0, 4]","[2, 3, 1]",0.0,0,0
4,Player 2,0,"[0, 4, 2, 1, 3]","[1, 3, 2]",0.0,0,0


The results are in and it looks like player 1 is winning 100% of the matches on a path of order 6. Let's try on a graph with order 9 and see how that affects the results.

In [9]:
p1_pop, p2_pop = path_test(num_vertices = 9, k_colors = 3)
print(p1_pop.head())
print(p2_pop.head())

        Name Gen                     Vertices     Colors  Fitness Wins Losses
11  Player 1   0  [6, 4, 8, 2, 3, 7, 1, 0, 5]  [1, 3, 2]      1.0  221      0
18  Player 1   0  [1, 0, 7, 6, 8, 3, 2, 4, 5]  [3, 1, 2]      1.0  221      0
52  Player 1   0  [6, 3, 5, 0, 1, 8, 4, 2, 7]  [3, 2, 1]      1.0  218      0
58  Player 1   0  [6, 0, 4, 7, 1, 2, 3, 8, 5]  [2, 3, 1]      1.0  217      0
71  Player 1   0  [8, 4, 1, 5, 2, 3, 6, 0, 7]  [1, 3, 2]      1.0  217      0
       Name Gen                     Vertices     Colors  Fitness Wins Losses
0  Player 2   0  [0, 2, 5, 8, 7, 4, 6, 1, 3]  [1, 2, 3]      0.0    0      0
1  Player 2   0  [4, 6, 2, 7, 3, 0, 1, 5, 8]  [3, 2, 1]      0.0    0      0
2  Player 2   0  [0, 7, 4, 5, 8, 2, 3, 6, 1]  [3, 2, 1]      0.0    0      0
3  Player 2   0  [5, 1, 8, 3, 0, 2, 7, 4, 6]  [3, 1, 2]      0.0    0      0
4  Player 2   0  [3, 5, 6, 8, 2, 1, 0, 7, 4]  [1, 2, 3]      0.0    0      0


In one last attempt, lets try a few larger odd-ordered graphs.

In [10]:
for order in [11, 33, 43]:
    print("\tTesting on path of order: " + str(order))
    p1_pop, p2_pop = path_test(num_vertices=order, k_colors=3)
    print(p1_pop.head())
    print(p2_pop.head())

	Testing on path of order: 11
        Name Gen                            Vertices     Colors  Fitness Wins  \
37  Player 1   0  [8, 3, 10, 5, 6, 2, 7, 0, 4, 1, 9]  [2, 1, 3]      1.0  239   
68  Player 1   0  [1, 5, 4, 9, 2, 6, 0, 10, 8, 7, 3]  [1, 2, 3]      1.0  222   
74  Player 1   0  [4, 0, 8, 10, 6, 2, 3, 9, 5, 7, 1]  [3, 2, 1]      1.0  222   
79  Player 1   0  [3, 8, 1, 6, 7, 9, 10, 2, 0, 5, 4]  [1, 2, 3]      1.0  219   
12  Player 1   0  [3, 1, 5, 2, 0, 4, 8, 10, 9, 7, 6]  [3, 2, 1]      1.0  218   

   Losses  
37      0  
68      0  
74      0  
79      0  
12      0  
       Name Gen                            Vertices     Colors  Fitness Wins  \
0  Player 2   0  [2, 8, 6, 1, 7, 5, 3, 4, 10, 9, 0]  [3, 1, 2]      0.0    0   
1  Player 2   0  [0, 1, 10, 2, 3, 5, 7, 9, 4, 6, 8]  [2, 3, 1]      0.0    0   
2  Player 2   0  [0, 8, 9, 10, 1, 3, 6, 4, 5, 2, 7]  [2, 1, 3]      0.0    0   
3  Player 2   0  [2, 0, 9, 10, 5, 4, 6, 3, 1, 8, 7]  [1, 2, 3]      0.0    0   
4  Player 2

Alright it seems that on an odd path with $k = 3$ is in fact a winning chromatic number. Lets run a few scenarios where we shift that value. We will test on the same size graphs to keep it consistent.

In [11]:
orders = [11, 33, 43]
k_colors = [2,3,4,5]

In [12]:
simulation1 = graph_simulation_suite(orders, k_colors, path_test)

----------------------------------------------------------------------

	Testing Graph of Order 11 with k = 2
       Name Gen                            Vertices  Colors  Fitness Wins  \
0  Player 1   0  [4, 7, 9, 8, 10, 3, 1, 5, 2, 6, 0]  [2, 1]      0.0    0   
1  Player 1   0  [1, 7, 8, 4, 2, 9, 10, 5, 6, 3, 0]  [2, 1]      0.0    0   
2  Player 1   0  [5, 6, 0, 2, 9, 1, 3, 4, 10, 7, 8]  [1, 2]      0.0    0   
3  Player 1   0  [6, 9, 0, 7, 10, 2, 1, 3, 8, 5, 4]  [2, 1]      0.0    0   
4  Player 1   0  [5, 3, 6, 2, 10, 4, 0, 8, 1, 7, 9]  [2, 1]      0.0    0   

  Losses  
0      0  
1      0  
2      0  
3      0  
4      0  
       Name Gen                            Vertices  Colors   Fitness Wins  \
1  Player 2   3  [2, 5, 6, 10, 1, 3, 4, 7, 9, 8, 0]  [2, 1]  1.000000   19   
0  Player 2   1  [1, 7, 4, 2, 0, 5, 3, 9, 8, 6, 10]  [1, 2]  1.000000   17   
2  Player 2   2  [2, 5, 6, 10, 7, 9, 8, 0, 3, 1, 4]  [1, 2]  0.984127  124   
3  Player 2   4  [1, 7, 4, 2, 5, 8, 10, 9, 0, 6, 

And now that we have collected all of those results in the variable `simulation1`, we can reference all the data frames later for further analysis. 

## Even Paths

Following the same process as for the odd path, we will now generate some data with regards to even paths. Let's start with $k = 3$ just like before and then we will move into the `graph_simulation_suite()`.

In [13]:
p1_pop, p2_pop = path_test(num_vertices = 4, k_colors = 3)

In [14]:
print(p1_pop.head())
print(p2_pop.head())

        Name Gen      Vertices     Colors  Fitness Wins Losses
62  Player 1   0  [0, 2, 3, 1]  [3, 2, 1]      1.0  225      0
90  Player 1   0  [2, 3, 1, 0]  [3, 2, 1]      1.0  224      0
68  Player 1   0  [3, 1, 0, 2]  [2, 3, 1]      1.0  220      0
4   Player 1   0  [0, 2, 1, 3]  [2, 1, 3]      1.0  216      0
28  Player 1   0  [0, 1, 2, 3]  [3, 1, 2]      1.0  216      0
       Name Gen      Vertices     Colors  Fitness Wins Losses
0  Player 2   0  [1, 0, 3, 2]  [3, 2, 1]      0.0    0      0
1  Player 2   0  [0, 3, 1, 2]  [3, 1, 2]      0.0    0      0
2  Player 2   0  [3, 2, 0, 1]  [3, 1, 2]      0.0    0      0
3  Player 2   0  [0, 2, 3, 1]  [3, 1, 2]      0.0    0      0
4  Player 2   0  [2, 3, 1, 0]  [1, 3, 2]      0.0    0      0


Lets shorten the path a bit

In [15]:
p1_pop, p2_pop = path_test(num_vertices = 2, k_colors = 3)
print(p1_pop.head())
print(p2_pop.head())

Error: Failed to create unique players 1000 times. Aborting
Error: Failed to create unique players 1000 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
        Name Gen Vertices     Colors  Fitness Wins Losses
0   Player 1   0   [0, 1]  [3, 2, 1]      1.0  214      0
8   Player 1   0   [0, 1]  [3, 1, 2]      1.0  208      0
9   Player 1   0   [0, 1]  [2, 1, 3]      1.0  207      0
3   Player 1   0   [1, 0]  [2, 1, 3]      1.0  206      0


An order of 2 (num_vertices = 2), appears to be a limitation of this current model and will be addressed in [Model Limitations](#Model-Limitations). 

Moving on to the test suite, defining the orders and k_colors to use in the model.

In [16]:
orders = [12, 24, 48]
k_colors = [2,3,4,5]
simulation2 = graph_simulation_suite(orders, k_colors, path_test);

----------------------------------------------------------------------

	Testing Graph of Order 12 with k = 2
       Name Gen                                Vertices  Colors  Fitness Wins  \
0  Player 1   0  [7, 9, 6, 2, 11, 0, 4, 8, 1, 5, 10, 3]  [2, 1]      0.0    0   
1  Player 1   0  [10, 3, 5, 2, 4, 0, 1, 7, 8, 11, 6, 9]  [2, 1]      0.0    0   
2  Player 1   0  [9, 4, 10, 1, 8, 3, 5, 6, 2, 11, 0, 7]  [1, 2]      0.0    0   
3  Player 1   0  [10, 0, 7, 8, 2, 4, 3, 11, 1, 5, 6, 9]  [1, 2]      0.0    0   
4  Player 1   0  [2, 4, 8, 10, 0, 3, 1, 7, 9, 6, 11, 5]  [2, 1]      0.0    0   

  Losses  
0      0  
1      0  
2      0  
3      0  
4      0  
       Name Gen                                Vertices  Colors   Fitness  \
0  Player 2   0  [7, 0, 4, 9, 2, 11, 1, 5, 8, 6, 3, 10]  [1, 2]  0.984772   
1  Player 2   0  [7, 2, 5, 10, 3, 6, 8, 1, 4, 9, 0, 11]  [2, 1]  0.975728   
2  Player 2   0  [3, 0, 6, 10, 11, 1, 4, 5, 8, 9, 7, 2]  [2, 1]  0.974227   
3  Player 2   0  [3, 10, 0, 7

And now that we have collected all of those results in the variable `simulation2`, we can reference all the data frames later for further analysis. 

-----
# Model Limitations
----

Every model is bound to have limitations and the few that I noticed were:

- When $k = 2$ for the Even Paths. This caused our Genetic Algorithm to crash or would cause the player populations would struggle to create players 
- Large graphs require lots of computation power and would require that they be run on cloud computing systems such as our own cloud server or Google collab.

---
# Conclusions
---

## Odd Path Review

From viewing the data in `simulation1` we can see:

- $k = 2$ meant player 1 lost 100% of the time
- $k = 3$ meant player 1 won 100% of the time
- $k = 4$ meant player 1 won 100% of the time
- $k = 5$ meant player 1 won 100% of the time



## Even Paths Review

From viewing the data in `simulation2` we can see:

- $k = 2$ meant player 1 lost 100% of the time
- $k = 3$ meant player 1 won 100% of the time
- $k = 4$ meant player 1 won 100% of the time
- $k = 5$ meant player 1 won 100% of the time

Noting that the game chromatic coloring (Minimal coloring) is $k = 3$, we have successfully shown this to hold true.